In [22]:
import requests
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import re
import urllib.request
import pandas as pd
import numpy as np
import time

In [2]:
def open_url_2_soup(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'}
    page = requests.get(url,headers = headers,verify = False)
    page_soup = soup(page.content,'lxml')
    temp = page_soup.findAll("div",{"class":"col-sm-9"})
#     print(temp[2].small.text)
    return page_soup

In [40]:

def getUniversityLookup(url):
    univ_name = []
    univ_link = []
    univ_soup = open_url_2_soup(url)
    univ_detail_tag = univ_soup.findAll("div",{"class":"col-sm-9 col-xs-12"})
    for tag in univ_detail_tag:
        a = tag.find("a",href=True)
        univ_name.append(a.text)
        univ_link.append(a['href'])
    #data cleaning 
    univ_df = pd.DataFrame({"name":np.asarray(univ_name),"href": np.asarray(univ_link)})
    print(univ_df['name'].str.split('(',n = 1, expand = True))
    univ_df[['name','keys']] = univ_df['name'].str.split('(',n = 1, expand = True)
    univ_df['keys'] = univ_df['keys'].apply(lambda x: x[:-1])
    return univ_df
# getUniversityLookup("https://yocket.in/universities?page=5")

In [51]:
def getUniversityList():
    UniversityList_df = pd.DataFrame()
    count = 1
    while(count <= 16):
        url = "https://yocket.in/universities?page=" + str(count)
        temp_df = getUniversityLookup(url)
#         print(temp_df)
        UniversityList_df = UniversityList_df.append(temp_df,ignore_index=True)
#         print(UniversityList_df)
        time.sleep(5) 
        count += 1
    return UniversityList_df




# open_url_2_soup("https://yocket.in/applications-admits-rejects/university-of-california-riverside/2")

In [52]:
def write_2_csv(df,name):
    df.to_csv(name + '.csv', encoding='utf-8')

In [ ]:
write_2_csv(getUniversityList(),'universityList')

C:\Users\lakshmib\AppData\Local\Continuum\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:791: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


                                               0                1
0                      Arizona State University              ASU)
1                       Northeastern University              NEU)
2         Texas A&M University, College Station             TAMU)
3             University of Southern California              USC)
4                 University of Texas at Dallas              UTD)
5              University of Texas at Arlington     UT Arlington)
6                    University at Buffalo SUNY     SUNY Buffalo)
7      North Carolina State University, Raleigh             NCSU)
8               Georgia Institute of Technology           GaTech)
9                 University of Texas at Austin        UT Austin)
10            University of Illinois at Chicago              UIC)
11                          New York University              NYU)
12             Illinois Institute of Technology      IIT Chicago)
13                    San Jose State University             SJSU)
14    Univ

C:\Users\lakshmib\AppData\Local\Continuum\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:791: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


                                          0             1
0                    Iowa State University    Iowa State)
1         Rutgers University-New Brunswick    Rutgers NB)
2                      Columbia University      Columbia)
3                    University of Houston    U. Houston)
4                Michigan State University      Mich. SU)
5      University of Massachusetts Amherst      Umass Am)
6                 University of Cincinnati      U.Cincin)
7           University of Colorado Boulder       Boulder)
8       New Jersey Institute of Technology          NJIT)
9          University of Wisconsin-Madison        UW Mad)
10                      Clemson University       Clemson)
11              University of Pennsylvania         UPenn)
12  California State University, Fullerton     Fullerton)
13                   Ohio State University           OSU)
14     University of California – Berkeley   UC Berkeley)
15         Stevens Institute of Technology    Stevens IT)
16            

C:\Users\lakshmib\AppData\Local\Continuum\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:791: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


...

                                                  0              1
0                         Dallas Bapist University            DBU)
1                               Adelphi University        adelphi)
2             Central Connecticut State University           CCSU)
3                                     Hood College           Hood)
4                               Hampton University        Hampton)
5                          Saint Xavier University            SXU)
6                             Fairfield University      Fairfield)
7                      University of North Alabama   UNA Florence)
8                         Norfolk State University        Norfolk)
9                   Winston-Salem State University           WSSU)
10                             La Salle University        LaSalle)
11  Saint Xavier University, Illinois- Orland Park     SXU Orland)
12                             Wesleyan University       Wesleyan)
13                  Christopher Newport University            

C:\Users\lakshmib\AppData\Local\Continuum\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:791: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


                                                    0                  1
0                                Duquesne University                DUQ)
1   Virginia Polytechnic Institute and State Unive...                VT)
2                      University of The Cumberlands        Cumberlands)
3                            Millersville University       Millersville)
4                                   La Roche College            Laroche)
5                          The College of Saint Rose             Strose)
6                          University of the Potomac            Potomac)
7                      Capitol Technology University           CapTechU)
8                            Keck Graduate Institute                KGI)
9           New York Film Academy South Beach Campus            NYFA SB)
10                           University of San Diego           sandiego)
11             Southern Polytechnic State University               SPSU)
12                                Ashland Universit

C:\Users\lakshmib\AppData\Local\Continuum\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:791: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


                                                    0              1
0                  Wentworth Institute of Technology            WIT)
1                         Texas Christian University            TCU)
2                              Willamette University     Willamette)
3                        University of Northern Iowa            UNI)
4                             University of Richmond       richmond)
5                           New York Medical College           NYMC)
6                                    Wheaton College        Wheaton)
7   University of Massachusetts Medical School Wor...      umassmed)
8                                 Yeshiva University             YU)
9                                    Emerson College        emerson)
10                             Mount Holyoke College       mtholoke)
11                              St Martin University       Stmartin)
12                            SIT Graduate Institute   SIT graduate)
13                                